# import các thư viện cần thiết

In [ ]:
import os
import random
import time
import itertools
import numpy as np
import pandas as pd

from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

import scipy.sparse as sp
from scipy.sparse import csr_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("PyTorch:", torch.__version__)

In [4]:
import random, numpy as np, torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)


In [5]:
import sys
import subprocess
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "fashion-clip"])

CompletedProcess(args=['/usr/bin/python3', '-m', 'pip', 'install', '-q', 'fashion-clip'], returncode=0)

In [6]:
from fashion_clip.fashion_clip import FashionCLIP

fclip = FashionCLIP('fashion-clip')

2025-12-11 10:00:31.596147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765447231.784950     362 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765447231.841821     362 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [7]:
def build_text_column(outfits: pd.DataFrame, cfg):
    text_cols = cfg.get("outfit_text_cols", [])
    def _build(row):
        parts = []
        for c in text_cols:
            if c in row and pd.notna(row[c]):
                parts.append(str(row[c]))
        return " ".join(parts)
    return outfits.apply(_build, axis=1)

In [ ]:
import os
import torch
import numpy as np

def save_cf_model(model_name, model, user_emb, item_emb, extra_info=None):
    """
    model_name: tên file, ví dụ 'ngcf', 'lightgcn', 'als'
    model: mô hình PyTorch (NGCF / LightGCN) hoặc None (cho ALS)
    user_emb, item_emb: torch.Tensor embedding
    extra_info: bất kỳ thông tin nào muốn lưu thêm (dict)
    """
    os.makedirs("saved_models", exist_ok=True)
    save_path = f"saved_models/{model_name}.pt"

    save_dict = {
        "user_emb": user_emb.cpu(),
        "item_emb": item_emb.cpu(),
        "extra": extra_info if extra_info is not None else {},
    }

    # Với NGCF / LightGCN thì có state_dict
    if model is not None:
        save_dict["model_state_dict"] = model.state_dict()

    torch.save(save_dict, save_path)
    print(f"[OK] Saved {model_name} → {save_path}")

In [40]:
def load_cf_model(model_name, ModelClass=None, device="cpu"):
    load_path = f"saved_models/{model_name}.pt"
    ckpt = torch.load(load_path, map_location=device)

    user_emb = ckpt["user_emb"].to(device)
    item_emb = ckpt["item_emb"].to(device)
    extra = ckpt["extra"]

    model = None
    if ModelClass is not None:
        # Detect model type automatically
        if ModelClass.__name__.lower() == "ngcf":
            model = ModelClass(
                extra["num_users"],
                extra["num_items"],
                dim=extra["dim"],
                layers=extra["layers"],
            ).to(device)

        elif ModelClass.__name__.lower() == "lightgcn":
            model = ModelClass(
                extra["num_users"],
                extra["num_items"],
                dim=extra["dim"],
                n_layers=extra["n_layers"],   # <--- dùng đúng tên tham số
            ).to(device)

        else:
            raise ValueError("ModelClass not supported.")

        model.load_state_dict(ckpt["model_state_dict"])
        model.eval()

    print(f"[OK] Loaded {model_name} from {load_path}")
    return model, user_emb, item_emb, extra


# Chuẩn bị dữ liệu H&M

In [ ]:
import os, numpy as np, pandas as pd
from tqdm import tqdm
from PIL import Image
from fashion_clip.fashion_clip import FashionCLIP

# =========================
# 0) Helper functions
# =========================
# rule H&M: folder = 3 số đầu, filename = 10 số article_id.jpg
def article_id_to_image_path(article_id, image_root):
    aid = str(article_id).zfill(10)
    folder = aid[:3]
    return os.path.join(image_root, folder, f"{aid}.jpg")

def load_dataset(base_path, cfg):
    sep = cfg.get("csv_sep", ",")
    outfits = pd.read_csv(
        os.path.join(base_path, cfg["outfits_file"]),
        sep=sep,
        quotechar='"',
        engine="python",
        on_bad_lines="skip",
        dtype={cfg["item_col"]: str}   # ✅ ép article_id thành str
    )
    user_activity = pd.read_csv(
        os.path.join(base_path, cfg["user_activity_file"]),
        sep=sep,
        quotechar='"',
        engine="python",
        dtype={cfg["item_col"]: str}
    )
    return outfits, user_activity

def build_group_images(outfits_df, cfg, image_dir):
    if "product_code" not in outfits_df.columns:
        raise ValueError("articles.csv không có cột product_code")

    records = []
    for _, row in outfits_df.iterrows():
        aid = row[cfg["item_col"]]   # str
        g   = row["product_code"]
        records.append({
            "product_code": g,
            "article_id": aid,
            "image_path": article_id_to_image_path(aid, image_dir)
        })
    return pd.DataFrame(records)

# =========================
# 1) Config H&M
# =========================
DATASET = "hm"
BASE_PATH = "/kaggle/input/h-and-m-personalized-fashion-recommendations"
IMAGE_DIR = os.path.join(BASE_PATH, "images")

EMB_DIR = "/kaggle/working/embeddings"
os.makedirs(EMB_DIR, exist_ok=True)

HM_TEXT_MAP_PATH  = os.path.join(EMB_DIR, f"{DATASET}_text_emb_map.npy")
HM_IMAGE_GRP_PATH = os.path.join(EMB_DIR, f"{DATASET}_image_emb_group.npy")
HM_FUSION_PATH    = os.path.join(EMB_DIR, f"{DATASET}_fusion_emb.npy")

CFG = {
    "outfits_file": "articles.csv",
    "user_activity_file": "transactions_train.csv",
    "csv_sep": ",",

    "user_col": "customer_id",
    "item_col": "article_id",
    "start_time_col": "t_dat",
    "end_time_col": "t_dat",

    "outfit_text_cols": [
        "prod_name",
        "product_type_name",
        "product_group_name",
        "graphical_appearance_name",
        "colour_group_name",
        "perceived_colour_value_name",
        "perceived_colour_master_name",
        "department_name",
        "index_name",
        "index_group_name",
        "section_name",
        "garment_group_name",
        "detail_desc",
    ],
}

print("BASE_PATH:", BASE_PATH)
print("IMAGE_DIR:", IMAGE_DIR)

# =========================
# 2) Load data
# =========================
outfits, user_activity = load_dataset(BASE_PATH, CFG)
print("Outfits (articles):", outfits.shape)
print("User activity:", user_activity.shape)

# =========================
# 3) Build group_images (product_code -> list ảnh)
# =========================
group_images = build_group_images(outfits, CFG, IMAGE_DIR)
print("group_images:", group_images.shape)
print(group_images.head())


In [ ]:
# from fashion_clip.fashion_clip import FashionCLIP
# fclip = FashionCLIP('fashion-clip')

# =========================
# TEXT-ONLY EMBEDDING (H&M) - PREFIX hm_
# =========================

DATASET_NAME = "hm"  # prefix cho tất cả file H&M

EMB_FILE_NAME = f"{DATASET_NAME}_text_only_embeddings.npy"
OUTFIT_EMB_PATH = os.path.join("/kaggle/working", EMB_FILE_NAME)

outfit_embeddings = None

if os.path.exists(OUTFIT_EMB_PATH):
    print(f"=> Đang load {DATASET_NAME} text-only embeddings từ", OUTFIT_EMB_PATH)
    arr = np.load(OUTFIT_EMB_PATH, allow_pickle=True)

    if isinstance(arr, np.ndarray) and arr.dtype == object and arr.size == 1:
        outfit_embeddings = arr.item()
        print("  - Loaded dict, số article:", len(outfit_embeddings))
    else:
        raise ValueError("Định dạng file embeddings không đúng (mong đợi dict).")

else:
    print(f"=> Không tìm thấy {EMB_FILE_NAME} → tính TEXT embedding cho toàn bộ articles...")

    outfits["__text_for_clip"] = build_text_column(outfits, CFG)
    texts = outfits["__text_for_clip"].fillna("").tolist()
    article_ids = outfits["article_id"].astype(str).values

    text_embeddings = fclip.encode_text(texts, batch_size=32)
    outfit_embeddings = dict(zip(article_ids, text_embeddings))

    print("=> Lưu text-only embeddings vào", OUTFIT_EMB_PATH)
    np.save(OUTFIT_EMB_PATH, outfit_embeddings)

print("Số article có embedding:", len(outfit_embeddings))
feat_dim = next(iter(outfit_embeddings.values())).shape[0]
print("Kích thước embedding:", feat_dim)

In [ ]:
import os
from glob import glob

IMAGE_DIR = "/kaggle/input/h-and-m-personalized-fashion-recommendations/images"

# Cách 1: đếm tất cả file .jpg trong toàn bộ thư mục con
all_imgs = glob(os.path.join(IMAGE_DIR, "*", "*.jpg"))
print("Tổng số ảnh (.jpg):", len(all_imgs))

# (tuỳ chọn) đếm theo từng folder 3 số đầu
folder_counts = {}
for folder in sorted(os.listdir(IMAGE_DIR)):
    fpath = os.path.join(IMAGE_DIR, folder)
    if os.path.isdir(fpath):
        folder_counts[folder] = len(glob(os.path.join(fpath, "*.jpg")))

print("Số folder:", len(folder_counts))
print("3 folder nhiều ảnh nhất:")
for k in sorted(folder_counts, key=folder_counts.get, reverse=True)[:3]:
    print(k, folder_counts[k])


# Chuẩn bị dữ liệu vibrent-clothes-rental-dataset

In [8]:
# ========= CONFIG DATASET =========
# Chỉ cần sửa block này khi đổi dataset

DATASET_NAME = "vibrent"  # tên để đặt file output embeddings, log,...

# local:
# BASE_PATH = r"D:\DACNTT"
# IMAGE_DIR = os.path.join(BASE_PATH, "images")

# Kaggle:
BASE_PATH = "/kaggle/input/vibrent-clothes-rental-dataset"
IMAGE_DIR = os.path.join(BASE_PATH, "images")
# File & cột tương ứng với dataset Vibrent
CFG = {
    "outfits_file": "outfits.csv",
    "pictures_file": "picture_triplets.csv",
    "user_activity_file": "user_activity_triplets.csv",
    "csv_sep": ";",
    # cột user / item / thời gian trong user_activity
    "user_col": "customer.id",
    "item_col": "outfit.id",
    "start_time_col": "rentalPeriod.start",
    "end_time_col": "rentalPeriod.end",
    # cột trong pictures
    "picture_outfit_col": "outfit.id",
    "picture_filename_col": "file_name",
    "picture_displayorder_col": "displayOrder",
    # cột để xây text cho FashionCLIP
    # sẽ join các cột này bằng khoảng trắng (bỏ NA)
    "outfit_text_cols": ["description", "outfit_tags"],
}

print("BASE_PATH:", BASE_PATH)
print("IMAGE_DIR:", IMAGE_DIR)
print("CFG:", CFG)


BASE_PATH: /kaggle/input/vibrent-clothes-rental-dataset
IMAGE_DIR: /kaggle/input/vibrent-clothes-rental-dataset/images
CFG: {'outfits_file': 'outfits.csv', 'pictures_file': 'picture_triplets.csv', 'user_activity_file': 'user_activity_triplets.csv', 'csv_sep': ';', 'user_col': 'customer.id', 'item_col': 'outfit.id', 'start_time_col': 'rentalPeriod.start', 'end_time_col': 'rentalPeriod.end', 'picture_outfit_col': 'outfit.id', 'picture_filename_col': 'file_name', 'picture_displayorder_col': 'displayOrder', 'outfit_text_cols': ['description', 'outfit_tags']}


load dữ liệu

In [9]:
def load_dataset(base_path, cfg):
    sep = cfg.get("csv_sep", ",")

    outfits = pd.read_csv(
        os.path.join(base_path, cfg["outfits_file"]),
        sep=sep,
        quotechar='"',
        engine="python",
        on_bad_lines="skip"
    )

    pictures = pd.read_csv(
        os.path.join(base_path, cfg["pictures_file"]),
        sep=sep,
        quotechar='"',
        engine="python"
    )

    user_activity = pd.read_csv(
        os.path.join(base_path, cfg["user_activity_file"]),
        sep=sep,
        quotechar='"',
        engine="python"
    )

    print("Outfits:", outfits.shape)
    print("Pictures:", pictures.shape)
    print("User activity:", user_activity.shape)
    return outfits, pictures, user_activity


outfits, pictures, user_activity = load_dataset(BASE_PATH, CFG)

Outfits: (15649, 11)
Pictures: (50193, 4)
User activity: (64419, 4)


In [24]:
image_dir = "/kaggle/input/vibrent-clothes-rental-dataset/images"

def load_image(file_name):
    path = os.path.join(image_dir, file_name)
    if os.path.exists(path):
        try:
            return Image.open(path)
        except:
            return None
    return None


In [26]:
main_pictures = (
    pictures.sort_values("displayOrder")
    .groupby("outfit.id")
    .first() 
    .reset_index()
)

main_pictures["image_obj"] = main_pictures["file_name"].apply(load_image)
main_pictures = main_pictures[main_pictures["image_obj"].notnull()]
print("Số outfit có main picture (theo CSV):", len(main_pictures))
print(main_pictures.head())

  0%|          | 0/4 [14:00<?, ?it/s]


Số outfit có main picture (theo CSV): 15157
                                 outfit.id  \
0  outfit.00004b4d01ca4ab0a70cf073ba74fefa   
1  outfit.0013691ff35b440e9dcfe1748ec184c7   
2  outfit.0014a5c89b244077a3d7cffd4549718e   
3  outfit.0018701ce6b049ebadc314d16623caa8   
4  outfit.001bf665330140cf854dcfb1cbff6b5f   

                                 picture.id  displayOrder  \
0  picture.a2b794c7ef83495a8997e7b0c318d65a             1   
1  picture.9c821ecbecb14c959f35078010fb91f3             1   
2  picture.b9aa39eb40f5410fa4fe101236241b19             1   
3  picture.b944a50f20fd4c7f954213dc7c38a776             1   
4  picture.fb1ff67a0bbc418b88ebb5560fac88a1             0   

                              file_name  \
0  a2b794c7ef83495a8997e7b0c318d65a.jpg   
1  9c821ecbecb14c959f35078010fb91f3.jpg   
2  b9aa39eb40f5410fa4fe101236241b19.jpg   
3  b944a50f20fd4c7f954213dc7c38a776.jpg   
4  fb1ff67a0bbc418b88ebb5560fac88a1.jpg   

                                           image_obj 

# Tạo outfitsEmbedding hoặc load lên nếu đã có

In [31]:
import os
import pickle
import numpy as np
import pandas as pd

EMB_FILE_NAME = "outfit_embeddings.pkl"
OUTFIT_EMB_PATH = os.path.join("/kaggle/working/", EMB_FILE_NAME)

outfit_embeddings = None


def build_text_column(outfits: pd.DataFrame, cfg):
    text_cols = cfg.get("outfit_text_cols", [])
    def _build(row):
        parts = []
        for c in text_cols:
            if c in row and pd.notna(row[c]):
                parts.append(str(row[c]))
        return " ".join(parts)
    return outfits.apply(_build, axis=1)


# ============================================================
# 1) LOAD EMBEDDING NẾU ĐÃ CÓ
# ============================================================
if os.path.exists(OUTFIT_EMB_PATH):
    print("=> Đang load outfit_embeddings từ", OUTFIT_EMB_PATH)

    with open(OUTFIT_EMB_PATH, "rb") as f:
        outfit_embeddings = pickle.load(f)

    print("  - Loaded dict, số outfit:", len(outfit_embeddings))

else:
    # ============================================================
    # 2) TÍNH TEXT + IMAGE EMBEDDING
    # ============================================================
    print("=> Không thấy embedding → bắt đầu tính TEXT + IMAGE bằng FashionCLIP")

    # ===== TEXT EMBEDDINGS =====
    print("[1/2] Encoding TEXT ...")
    outfits["text"] = build_text_column(outfits, CFG)
    text_embeddings = fclip.encode_text(outfits["text"].tolist(), batch_size=32)

    # ===== IMAGE EMBEDDINGS =====
    print("[2/2] Encoding IMAGES ...")
    images = main_pictures["image_obj"].tolist()
    image_embeddings = fclip.encode_images(images, batch_size=32)
    print("Ảnh encode được:", len(image_embeddings))

    # ===== GỘP TEXT + IMAGE =====
    text_emb_map = dict(zip(outfits["id"], text_embeddings))
    image_emb_map = dict(zip(main_pictures["outfit.id"], image_embeddings))

    outfit_embeddings = {}

    for oid in outfits["id"]:
        if oid in text_emb_map and oid in image_emb_map:
            t = text_emb_map[oid]
            i = image_emb_map[oid]

            # Chuẩn hóa trước khi cộng
            t = t / np.linalg.norm(t)
            i = i / np.linalg.norm(i)

            outfit_embeddings[oid] = 0.5 * t + 0.5 * i

    # ===== SAVE =====
    print("=> Saving:", OUTFIT_EMB_PATH)
    with open(OUTFIT_EMB_PATH, "wb") as f:
        pickle.dump(outfit_embeddings, f)

# ============================================================
# 3) THÔNG TIN EMBEDDING
# ============================================================
print("Done. Outfit embeddings =", len(outfit_embeddings))
feat_dim = next(iter(outfit_embeddings.values())).shape[0]
print("Embedding dimension =", feat_dim)


=> Đang load outfit_embeddings từ /kaggle/working/outfit_embeddings.pkl
  - Loaded dict, số outfit: 15157
Done. Outfit embeddings = 15157
Embedding dimension = 512


# Hàm lấy outfits từ text 

In [32]:
import numpy as np

def find_outfits_from_text(query_text, outfit_embeddings, model, topk=10):
    """
    Tìm top-K outfit phù hợp với text (query) dựa trên FashionCLIP embedding.
    """
    # 1) Encode text thành embedding
    text_emb = model.encode_text([query_text], batch_size=1)[0]
    text_emb = text_emb.astype(np.float32)

    # L2 normalize để cosine similarity
    text_emb = text_emb / (np.linalg.norm(text_emb) + 1e-8)

    # 2) Chuẩn bị item embedding matrix
    outfit_ids = list(outfit_embeddings.keys())
    item_matrix = np.vstack([outfit_embeddings[oid] for oid in outfit_ids]).astype(np.float32)

    # Normalize item embeddings
    item_matrix = item_matrix / (np.linalg.norm(item_matrix, axis=1, keepdims=True) + 1e-8)

    # 3) Tính cosine similarity
    scores = item_matrix @ text_emb     # shape = [num_items]

    # 4) Lấy top-k
    top_idx = np.argsort(-scores)[:topk]

    results = [(outfit_ids[i], float(scores[i])) for i in top_idx]
    return results
def get_images_for_query(query_text, topk=5):
    results = find_outfits_from_text(query_text, outfit_embeddings, fclip, topk=topk)

    pics = []
    for oid, score in results:
        row = main_pictures[main_pictures["outfit.id"] == oid]
        if len(row) > 0:
            file = row.iloc[0]["file_name"]
            img = load_image(file)
            pics.append((img, oid, score))
    return pics


In [ ]:
from IPython.display import display

query = "red dress"
pics = get_images_for_query(query, topk=5)

for img, oid, score in pics:
    print(f"Outfit: {oid} | score={score:.3f}")
    display(img)


# BERT+ImageEmbedding có sẵn

In [ ]:
# import sys
# import subprocess
# subprocess.run([sys.executable, "-m", "pip", "install", "-q", "transformers"])

In [ ]:
# from transformers import BertTokenizer, BertModel
# import torch
# from tqdm import tqdm

# device = "cuda" if torch.cuda.is_available() else "cpu"

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# bert_model = BertModel.from_pretrained("bert-base-uncased").to(device)
# bert_model.eval()

# def encode_texts_bert(texts, batch_size=16):
#     embeddings = []
#     for i in range(0, len(texts), batch_size):
#         batch = texts[i:i+batch_size]
#         tokens = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=64).to(device)
#         with torch.no_grad():
#             outputs = bert_model(**tokens)
#             cls_emb = outputs.pooler_output  # vector [CLS]
#             cls_emb = torch.nn.functional.normalize(cls_emb, p=2, dim=1)
#             embeddings.append(cls_emb.cpu().numpy())
#     return np.vstack(embeddings)

# # === Tạo embedding văn bản bằng BERT ===
# texts = outfits["text"].fillna("").tolist()
# bert_text_embeddings = encode_texts_bert(texts, batch_size=16)
# print("BERT embedding shape:", bert_text_embeddings.shape)



In [ ]:
# embedding_dir = os.path.join(base_path, "embeddings", "EfficientNet_V2_L_final")
# dataset_image_embeddings = {}
# for _, row in main_pictures.iterrows():
#     picture_id = row["picture.id"]
#     file_path = os.path.join(embedding_dir, f"picture.{picture_id}.npy")
#     if os.path.exists(file_path):
#         try:
#             vec = np.load(file_path)
#             dataset_image_embeddings[row["outfit.id"]] = vec
#         except Exception as e:
#             print(f"Lỗi load {file_path}: {e}")

# CFRS 

In [ ]:
#H&M
#CF preprocessing – encode user/item, split train/test, build A_train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

u_col = CFG["user_col"]
i_col = CFG["item_col"]
t_start = CFG["start_time_col"]
t_end = CFG["end_time_col"]

# 1) Parse thời gian & sort
for col in [t_start, t_end]:
    user_activity[col] = pd.to_datetime(user_activity[col], errors="coerce", utc=True)
user_activity = user_activity.dropna(subset=[t_start, t_end])
user_activity = user_activity.sort_values([u_col, t_start, t_end])
print("[OK] Parse & sort thời gian.")

# === CẮT TOP 10% USER HOẠT ĐỘNG NHẤT ===
user_counts = user_activity[u_col].value_counts()
ratio = 0.10
n_users_keep = max(1, int(len(user_counts) * ratio))
keep_users = user_counts.head(n_users_keep).index

user_activity = user_activity[user_activity[u_col].isin(keep_users)].copy()

print("[AFTER CUT 10%] users =", user_activity[u_col].nunique(),
      "| items =", user_activity[i_col].nunique(),
      "| interactions =", len(user_activity))

# 2) Encode user / item
user_enc = LabelEncoder().fit(user_activity[u_col].astype(str))
item_enc = LabelEncoder().fit(user_activity[i_col].astype(str))

user_activity["user_id"] = user_enc.transform(user_activity[u_col].astype(str))
user_activity["item_id"] = item_enc.transform(user_activity[i_col].astype(str))

num_users = user_activity["user_id"].nunique()
num_items = user_activity["item_id"].nunique()

# mapping giữa item_id (index trong matrix) và outfit-id gốc
oid_from_item = dict(zip(user_activity["item_id"], user_activity[i_col]))
item_from_oid = {v: k for k, v in oid_from_item.items()}

print(f"[COUNTS] users={num_users} | items={num_items} | interactions={len(user_activity)}")

# 3) Chia train/test theo thời gian cho từng user
TEST_RATIO = 0.2
user_train_items, user_test_items = {}, {}

for u, g in user_activity.groupby("user_id", sort=False):
    g = g.sort_values([t_start, t_end])
    items = list(g["item_id"])
    if len(items) < 3:
        continue
    test_size = max(1, int(len(items) * TEST_RATIO))
    user_train_items[u] = items[:-test_size]
    user_test_items[u] = items[-test_size:]

print(f"[SPLIT] users trong split = {len(user_train_items)}")

# Loại user không còn test hợp lệ
train_item_set = set(i for items in user_train_items.values() for i in items)
for u in list(user_test_items.keys()):
    filtered = [i for i in user_test_items[u] if i in train_item_set]
    if len(filtered) == 0:
        user_test_items.pop(u, None)
        user_train_items.pop(u, None)
    else:
        user_test_items[u] = filtered

print(f"[SPLIT after filter] users = {len(user_train_items)}")

# 4) Build A_train (CSR)
rows, cols = [], []
for u, items in user_train_items.items():
    rows.extend([u] * len(items))
    cols.extend(items)

if not rows:
    raise RuntimeError("Không còn tương tác train nào sau khi split.")

A_train = csr_matrix(
    (np.ones(len(rows), dtype=np.float32),
     (np.array(rows), np.array(cols))),
    shape=(num_users, num_items),
    dtype=np.float32
)
A_train.data[:] = 1.0
A_train.eliminate_zeros()

print(f"[MATRIX] A_train shape={A_train.shape} | nnz={A_train.nnz}")


In [44]:
#vibrent
#CF preprocessing – encode user/item, split train/test, build A_train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

u_col = CFG["user_col"]
i_col = CFG["item_col"]
t_start = CFG["start_time_col"]
t_end = CFG["end_time_col"]

# 1) Parse thời gian & sort
for col in [t_start, t_end]:
    user_activity[col] = pd.to_datetime(user_activity[col], errors="coerce", utc=True)
user_activity = user_activity.dropna(subset=[t_start, t_end])
user_activity = user_activity.sort_values([u_col, t_start, t_end])
print("[OK] Parse & sort thời gian.")

# 2) Encode user / item
user_enc = LabelEncoder().fit(user_activity[u_col].astype(str))
item_enc = LabelEncoder().fit(user_activity[i_col].astype(str))

user_activity["user_id"] = user_enc.transform(user_activity[u_col].astype(str))
user_activity["item_id"] = item_enc.transform(user_activity[i_col].astype(str))

num_users = user_activity["user_id"].nunique()
num_items = user_activity["item_id"].nunique()

# mapping giữa item_id (index trong matrix) và outfit-id gốc
oid_from_item = dict(zip(user_activity["item_id"], user_activity[i_col]))
item_from_oid = {v: k for k, v in oid_from_item.items()}

print(f"[COUNTS] users={num_users} | items={num_items} | interactions={len(user_activity)}")

# 3) Chia train/test theo thời gian cho từng user
TEST_RATIO = 0.2
user_train_items, user_test_items = {}, {}

for u, g in user_activity.groupby("user_id", sort=False):
    g = g.sort_values([t_start, t_end])
    items = list(g["item_id"])
    if len(items) < 3:
        continue
    test_size = max(1, int(len(items) * TEST_RATIO))
    user_train_items[u] = items[:-test_size]
    user_test_items[u] = items[-test_size:]

print(f"[SPLIT] users trong split = {len(user_train_items)}")

# Loại user không còn test hợp lệ
train_item_set = set(i for items in user_train_items.values() for i in items)
for u in list(user_test_items.keys()):
    filtered = [i for i in user_test_items[u] if i in train_item_set]
    if len(filtered) == 0:
        user_test_items.pop(u, None)
        user_train_items.pop(u, None)
    else:
        user_test_items[u] = filtered

print(f"[SPLIT after filter] users = {len(user_train_items)}")

# 4) Build A_train (CSR)
rows, cols = [], []
for u, items in user_train_items.items():
    rows.extend([u] * len(items))
    cols.extend(items)

if not rows:
    raise RuntimeError("Không còn tương tác train nào sau khi split.")

A_train = csr_matrix(
    (np.ones(len(rows), dtype=np.float32),
     (np.array(rows), np.array(cols))),
    shape=(num_users, num_items),
    dtype=np.float32
)
A_train.data[:] = 1.0
A_train.eliminate_zeros()

print(f"[MATRIX] A_train shape={A_train.shape} | nnz={A_train.nnz}")


Device: cuda
[OK] Parse & sort thời gian.
[COUNTS] users=2293 | items=10986 | interactions=64419
[SPLIT] users trong split = 1979
[SPLIT after filter] users = 1956
[MATRIX] A_train shape=(2293, 10986) | nnz=47356


In [45]:
#Các hàm cần thiết
class BPRLoss(nn.Module):
    def __init__(self, reg_lambda=1e-4):
        super().__init__()
        self.reg_lambda = reg_lambda

    def forward(self, user_emb, pos_emb, neg_emb):
        pos_score = torch.sum(user_emb * pos_emb, dim=1)
        neg_score = torch.sum(user_emb * neg_emb, dim=1)
        diff = pos_score - neg_score
        bpr = F.softplus(-diff).mean()
        reg = (
            user_emb.norm(2).pow(2)
            + pos_emb.norm(2).pow(2)
            + neg_emb.norm(2).pow(2)
        ) / user_emb.size(0)
        return bpr + self.reg_lambda * reg


def l2n_t(x, eps=1e-8):
    return x / (x.norm(dim=1, keepdim=True) + eps)


def sample_triplets(user_train_items, num_items, n_samples_per_user=5):
    users, pos_items, neg_items = [], [], []
    for u, pos_list in user_train_items.items():
        if not pos_list:
            continue
        pos_set = set(pos_list)
        for _ in range(n_samples_per_user):
            p = random.choice(pos_list)
            n = np.random.randint(0, num_items)
            while n in pos_set:
                n = np.random.randint(0, num_items)
            users.append(u)
            pos_items.append(p)
            neg_items.append(n)
    if not users:
        users, pos_items, neg_items = [0], [0], [1]
    return (
        torch.tensor(users, dtype=torch.long),
        torch.tensor(pos_items, dtype=torch.long),
        torch.tensor(neg_items, dtype=torch.long),
    )


@torch.no_grad()
def build_item_neighbors(item_emb, topk=100, batch=2048):
    item_emb = l2n_t(item_emb)
    N, _ = item_emb.shape
    all_topk = []
    for i0 in range(0, N, batch):
        xb = item_emb[i0 : i0 + batch]
        scores = xb @ item_emb.T
        for b in range(scores.shape[0]):
            idx = i0 + b
            if idx < N:
                scores[b, idx] = -1e9
        _, idxs = torch.topk(scores, k=min(topk, N - 1), dim=1)
        all_topk.append(idxs.cpu().numpy())
    return np.vstack(all_topk)


def hard_negative_sampling_from_neighbors(user_train_items, neighbors_idx, n_samples_per_user=5):
    users, pos_items, neg_items = [], [], []
    for u, pos_list in user_train_items.items():
        if not pos_list:
            continue
        pos_set = set(pos_list)
        for _ in range(n_samples_per_user):
            p = random.choice(pos_list)
            cand = neighbors_idx[p]
            cand = [c for c in cand if c not in pos_set]
            if not cand:
                continue
            n = random.choice(cand)
            users.append(u)
            pos_items.append(p)
            neg_items.append(n)
    if not users:
        return sample_triplets(user_train_items, num_items, n_samples_per_user)
    return (
        torch.tensor(users, dtype=torch.long),
        torch.tensor(pos_items, dtype=torch.long),
        torch.tensor(neg_items, dtype=torch.long),
    )


def mixed_negative_sampling(user_train_items, num_items, item_emb=None, neighbors_idx=None,
                            n_samples_per_user=10, mix=0.5):
    if (item_emb is not None) and (neighbors_idx is not None) and (np.random.rand() < mix):
        return hard_negative_sampling_from_neighbors(user_train_items, neighbors_idx, n_samples_per_user)
    else:
        return sample_triplets(user_train_items, num_items, n_samples_per_user)


def build_norm_adj_sparse(A_csr: csr_matrix) -> sp.csr_matrix:
    n_users, n_items = A_csr.shape
    R = A_csr.tocsr()
    upper = sp.hstack([sp.csr_matrix((n_users, n_users), dtype=np.float32), R], format="csr")
    lower = sp.hstack([R.T, sp.csr_matrix((n_items, n_items), dtype=np.float32)], format="csr")
    adj = sp.vstack([upper, lower], format="csr").astype(np.float32)

    deg = np.array(adj.sum(axis=1)).flatten().astype(np.float32)
    deg_inv_sqrt = np.zeros_like(deg, dtype=np.float32)
    mask = deg > 0
    deg_inv_sqrt[mask] = np.power(deg[mask], -0.5, dtype=np.float32)
    D_inv_sqrt = sp.diags(deg_inv_sqrt, format="csr", dtype=np.float32)

    norm_adj = D_inv_sqrt @ adj @ D_inv_sqrt
    return norm_adj.tocsr()


def scipy_to_torch_sparse(mat: sp.csr_matrix) -> torch.Tensor:
    coo = mat.tocoo()
    idx = np.vstack([coo.row, coo.col]).astype(np.int64)
    indices = torch.from_numpy(idx)
    values = torch.from_numpy(coo.data.astype(np.float32))
    return torch.sparse_coo_tensor(indices, values, coo.shape).coalesce()


def evaluate_embeddings(user_emb, item_emb, user_train_items, user_test_items, k=10):
    U = l2n_t(user_emb.cpu())
    I = l2n_t(item_emb.cpu())
    I_np = I.numpy()

    precisions, recalls, ndcgs, hit_rates = [], [], [], []

    for u in user_test_items.keys():
        train_items = set(user_train_items.get(u, []))
        test_items = set(user_test_items.get(u, []))
        if not test_items:
            continue

        scores = cosine_similarity(U[u].unsqueeze(0).numpy(), I_np)[0]
        # if train_items:
        #     scores[list(train_items)] = -1e9

        top_k = np.argpartition(scores, -k)[-k:]
        top_k = top_k[np.argsort(scores[top_k])[::-1]]

        hits = len(set(top_k) & test_items)

        # ---- Recall/Precision như cũ ----
        precision = hits / k
        recall = hits / len(test_items)

        # ---- NDCG như cũ ----
        dcg = sum(1 / np.log2(i + 2) for i, iid in enumerate(top_k) if iid in test_items)
        idcg = sum(1 / np.log2(i + 2) for i in range(min(len(test_items), k)))
        ndcg = dcg / idcg if idcg > 0 else 0

        # ---- HitRate@K = 1 nếu có ít nhất 1 hit ----
        hit = 1.0 if hits > 0 else 0.0

        precisions.append(precision)
        recalls.append(recall)
        ndcgs.append(ndcg)
        hit_rates.append(hit)

    return {
        f"Precision@{k}": float(np.mean(precisions)),
        f"Recall@{k}": float(np.mean(recalls)),
        f"NDCG@{k}": float(np.mean(ndcgs)),
        f"HitRate@{k}": float(np.mean(hit_rates)),
        "Users_eval": int(len(precisions)),
    }



In [46]:
# Item–item similarity từ FashionCLIP + build norm_adj_dev
feat_dim = next(iter(outfit_embeddings.values())).shape[0]
item_feat = np.zeros((num_items, feat_dim), dtype=np.float32)
has_emb = np.zeros(num_items, dtype=bool)

for item_id, oid in oid_from_item.items():
    v = outfit_embeddings.get(oid)
    if v is not None:
        item_feat[item_id] = v.astype(np.float32)
        has_emb[item_id] = True

print("Số item có embedding CLIP:", has_emb.sum(), "/", num_items)

item_feat = item_feat / (np.linalg.norm(item_feat, axis=1, keepdims=True) + 1e-8)

# Dense similarity (sau đó sparse hóa top-K)
# sim_item_item = item_feat @ item_feat.T
# print("Shape sim_item_item:", sim_item_item.shape)

# K = 50
# rows, cols, vals = [], [], []
# for i in range(num_items):
#     row = sim_item_item[i]
#     topk_idx = np.argpartition(-row, min(K + 1, num_items - 1))[: K + 1]
#     topk_idx = topk_idx[topk_idx != i]
#     topk_idx = topk_idx[:K]
#     for j in topk_idx:
#         sim_ij = float(row[j])
#         if sim_ij > 0:
#             rows.append(i)
#             cols.append(j)
#             vals.append(sim_ij)

# S_sparse = csr_matrix((vals, (rows, cols)), shape=(num_items, num_items), dtype=np.float32)
# print("S_sparse: shape =", S_sparse.shape,
#       "| nnz =", S_sparse.nnz,
#       "| avg neighbors per item ~", S_sparse.nnz / num_items)

# Filter user + remap, build A_train_new và norm_adj_dev (tái dùng giữa dataset)
MIN_TRAIN = 3
MIN_TEST = 1

kept_users = [
    u for u in user_train_items.keys()
    if len(user_train_items[u]) >= MIN_TRAIN and len(user_test_items.get(u, [])) >= MIN_TEST
]
print(f"[FILTER] kept users: {len(kept_users)} / {len(user_train_items)}")

old2new_user = {u_old: i for i, u_old in enumerate(sorted(kept_users))}
new2old_user = {i: u_old for u_old, i in old2new_user.items()}
num_users_new = len(old2new_user)

user_train_items_new, user_test_items_new = {}, {}
for u_old in kept_users:
    u_new = old2new_user[u_old]
    user_train_items_new[u_new] = list(user_train_items[u_old])
    user_test_items_new[u_new] = list(user_test_items[u_old])

rows, cols = [], []
for u_new, items in user_train_items_new.items():
    rows.extend([u_new] * len(items))
    cols.extend(items)

A_train_new = csr_matrix(
    (np.ones(len(rows), dtype=np.float32), (np.array(rows), np.array(cols))),
    shape=(num_users_new, num_items),
    dtype=np.float32
)
A_train_new.data[:] = 1.0
A_train_new.eliminate_zeros()
print(f"[MATRIX] A_train_new shape={A_train_new.shape} | nnz={A_train_new.nnz}")

# lambda_s = 0.3
# A_train_soft = A_train_new @ S_sparse
# A_train_soft = A_train_new + lambda_s * A_train_soft
# A_train_soft.eliminate_zeros()

A_for_adj = A_train_new      
# A_for_adj = A_train_soft

norm_adj_sp_new = build_norm_adj_sparse(A_for_adj)
norm_adj_new = scipy_to_torch_sparse(norm_adj_sp_new)

user_train_items = user_train_items_new
user_test_items = user_test_items_new
A_train = A_train_new
num_users = num_users_new

norm_adj_dev = norm_adj_new.to(device)
print("[READY] norm_adj_dev shape:", norm_adj_dev.shape,
      "| num_users + num_items =", num_users + num_items)


Số item có embedding CLIP: 10980 / 10986
[FILTER] kept users: 1828 / 1956
[MATRIX] A_train_new shape=(1828, 10986) | nnz=47102
[READY] norm_adj_dev shape: torch.Size([12814, 12814]) | num_users + num_items = 12814


# NGCF

In [36]:
class NGCF(nn.Module):
    def __init__(self, n_users, n_items, dim=128, layers=1, dropout=0.2):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.dim = dim
        self.layers = layers

        self.embedding = nn.Embedding(n_users + n_items, dim)
        nn.init.xavier_uniform_(self.embedding.weight)

        self.W1 = nn.ModuleList([nn.Linear(dim, dim) for _ in range(layers)])
        self.W2 = nn.ModuleList([nn.Linear(dim, dim) for _ in range(layers)])
        self.dropout = nn.Dropout(dropout)

    def forward(self, norm_adj):
        x = self.embedding.weight
        embs = [x]
        for k in range(self.layers):
            side = torch.sparse.mm(norm_adj, x)
            sum_emb = self.W1[k](side + x)
            bi_emb = self.W2[k](side * x)
            x = F.leaky_relu(sum_emb + bi_emb, 0.2)
            x = self.dropout(x)
            x = F.normalize(x, dim=1)
            embs.append(x)
        return torch.cat(embs, dim=1)

In [35]:
#H&M
# ================== TRAIN NGCF (phiên bản nhẹ hơn) ==================
ngcf = NGCF(num_users, num_items, dim=64, layers=1, dropout=0.1).to(device)
optimizer = torch.optim.AdamW(ngcf.parameters(), lr=0.005, weight_decay=1e-4)
bpr = BPRLoss(reg_lambda=1e-4)

epochs = 100              # giảm bớt cho đỡ nặng, sau chọn config tốt thì train lâu hơn
refresh_every = 10        # update hard negative mỗi 10 epoch
n_samples_per_user = 20
hard_ratio = 0.3
eval_every = 5

loss_history_NGCF = []
neighbors_idx = None

print("Training NGCF (light) ...")
for epoch in range(epochs):
    ngcf.train()

    # 1) Forward 1 lần để lấy embedding (TRONG chế độ train)
    emb_all = ngcf(norm_adj_dev)         # [U+I, D_all]
    u_emb = emb_all[:num_users]
    i_emb = emb_all[num_users:]

    # 2) Build neighbors cho hard negative (ít thường xuyên hơn + nhẹ hơn)
    with torch.no_grad():
        if (epoch == 0) or (epoch % refresh_every == 0):
            # dùng bản detach để không giữ graph
            item_emb_cache = i_emb.detach()
            neighbors_idx = build_item_neighbors(
                item_emb_cache,
                topk=50,     # giảm từ 100 -> 50
                batch=512,   # giảm batch cho đỡ tốn RAM
            )

    # 3) Negative sampling
    users, pos_items, neg_items = mixed_negative_sampling(
        user_train_items,
        num_items,
        item_emb=i_emb.detach(),         # dùng embedding detach hiện tại
        neighbors_idx=neighbors_idx,
        n_samples_per_user=n_samples_per_user,
        mix=hard_ratio,
    )
    users = users.to(device)
    pos_items = pos_items.to(device)
    neg_items = neg_items.to(device)

    user_batch_emb = u_emb[users]
    pos_emb = i_emb[pos_items]
    neg_emb = i_emb[neg_items]

    loss = bpr(user_batch_emb, pos_emb, neg_emb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_history_NGCF.append(loss.item())

    # 4) Eval định kỳ (dùng eval mode, forward riêng nhưng không phải mỗi epoch)
    if (epoch + 1) % eval_every == 0 or epoch == epochs - 1:
        ngcf.eval()
        with torch.no_grad():
            emb_eval = ngcf(norm_adj_dev)
            u_eval = emb_eval[:num_users]
            i_eval = emb_eval[num_users:]
            metrics = evaluate_embeddings(
                u_eval, i_eval, user_train_items, user_test_items, k=10
            )
        print(
            f"Epoch {epoch+1:03d} | loss={loss.item():.4f} | "
            f"HR@10={metrics['HitRate@10']:.4f} | "
            f"Recall@10={metrics['Recall@10']:.4f} | "
            f"NDCG@10={metrics['NDCG@10']:.4f} | "
            f"Precision@10={metrics['Precision@10']:.4f} | "
            f"Users_eval={metrics['Users_eval']}"
        )

ngcf.eval()
with torch.no_grad():
    emb_final = ngcf(norm_adj_dev)
    user_emb_ngcf = emb_final[:num_users]
    item_emb_ngcf = emb_final[num_users:]

print("Done. user_emb_ngcf:", user_emb_ngcf.shape,
      "| item_emb_ngcf:", item_emb_ngcf.shape)

NameError: name 'NGCF' is not defined

In [ ]:
ngcf = NGCF(num_users, num_items, dim=128, layers=2, dropout=0.1).to(device)
optimizer = torch.optim.AdamW(ngcf.parameters(), lr=0.01, weight_decay=1e-4)
bpr = BPRLoss(reg_lambda=1e-4)

epochs = 200
refresh_every = 5
n_samples_per_user = 20
hard_ratio = 0.3

loss_history_NGCF = []
neighbors_idx = None

print("Training NGCF ...")
for epoch in range(epochs):
    ngcf.train()

    with torch.no_grad():
        emb_cache = ngcf(norm_adj_dev)
        item_emb_cache = emb_cache[num_users:]
        if (epoch == 0) or (epoch % refresh_every == 0):
            neighbors_idx = build_item_neighbors(item_emb_cache, topk=100, batch=1024)

    users, pos_items, neg_items = mixed_negative_sampling(
        user_train_items,
        num_items,
        item_emb=item_emb_cache,
        neighbors_idx=neighbors_idx,
        n_samples_per_user=n_samples_per_user,
        mix=hard_ratio,
    )
    users = users.to(device)
    pos_items = pos_items.to(device)
    neg_items = neg_items.to(device)

    emb = ngcf(norm_adj_dev)
    u_emb = emb[:num_users]
    i_emb = emb[num_users:]

    user_batch_emb = u_emb[users]
    pos_emb = i_emb[pos_items]
    neg_emb = i_emb[neg_items]

    loss = bpr(user_batch_emb, pos_emb, neg_emb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_history_NGCF.append(loss.item())

    if (epoch + 1) % 5  == 0:
        ngcf.eval()
        with torch.no_grad():
            emb_eval = ngcf(norm_adj_dev)
            u_eval = emb_eval[:num_users]
            i_eval = emb_eval[num_users:]
            metrics = evaluate_embeddings(
                u_eval, i_eval, user_train_items, user_test_items, k=10
            )
        print(
            f"Epoch {epoch+1:03d} | loss={loss.item():.4f} | "
            f"HR@10={metrics['HitRate@10']:.4f} | "
            f"Recall@10={metrics['Recall@10']:.4f} | "
            f"NDCG@10={metrics['NDCG@10']:.4f} | "
            f"Precision@10={metrics['Precision@10']:.4f} | "
            f"Users_eval={metrics['Users_eval']}"
        )

ngcf.eval()
with torch.no_grad():
    emb_final = ngcf(norm_adj_dev)
    user_emb_ngcf = emb_final[:num_users]
    item_emb_ngcf = emb_final[num_users:]

print("Done. user_emb_ngcf:", user_emb_ngcf.shape,
      "| item_emb_ngcf:", item_emb_ngcf.shape)

In [ ]:
import os
os.makedirs("saved_models", exist_ok=True)

save_path = "saved_models/ngcf_model.pt"

torch.save({
    "model_state_dict": ngcf.state_dict(),
    "num_users": num_users,
    "num_items": num_items,
    "dim": ngcf.dim,
    "layers": ngcf.layers,
    "user_emb": user_emb_ngcf.cpu(),
    "item_emb": item_emb_ngcf.cpu(),
}, save_path)

print("Saved NGCF model to:", save_path)

In [37]:
checkpoint = torch.load("saved_models/ngcf_model.pt", map_location=device)

# Khởi tạo model với đúng kiến trúc
ngcf_loaded = NGCF(
    checkpoint["num_users"],
    checkpoint["num_items"],
    dim=checkpoint["dim"],
    layers=checkpoint["layers"],
    dropout=0.1
).to(device)

ngcf_loaded.load_state_dict(checkpoint["model_state_dict"])
ngcf_loaded.eval()

print("Loaded NGCF model.")
user_emb_ngcf = checkpoint["user_emb"].to(device)
item_emb_ngcf = checkpoint["item_emb"].to(device)


Loaded NGCF model.


# LightGCN

In [38]:
# ================== LIGHTGCN MODEL ==================
class LightGCN(nn.Module):
    def __init__(self, n_users, n_items, dim=128, n_layers=3):
        super().__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.dim = dim
        self.n_layers = n_layers

        # 1 embedding chung cho user + item
        self.embedding = nn.Embedding(n_users + n_items, dim)
        nn.init.xavier_uniform_(self.embedding.weight)

    def forward(self, norm_adj):
        """
        norm_adj: torch.sparse_coo_tensor (user+item graph)
        return: embedding cuối cùng (user+item) sau khi average các layer
        """
        all_embs = []
        x = self.embedding.weight     # [n_users+n_items, dim]
        all_embs.append(x)

        for _ in range(self.n_layers):
            x = torch.sparse.mm(norm_adj, x)   # propagate
            x = F.normalize(x, dim=1)
            all_embs.append(x)

        # stack [n_nodes, n_layers+1, dim] rồi mean theo layer
        embs = torch.stack(all_embs, dim=1)
        out = torch.mean(embs, dim=1)
        out = F.normalize(out, dim=1)
        return out


In [ ]:
# ================== TRAIN LIGHTGCN ==================
lightgcn = LightGCN(num_users, num_items, dim=128, n_layers=3).to(device)
optimizer_lg = torch.optim.AdamW(lightgcn.parameters(), lr=0.01, weight_decay=1e-4)
bpr_lg = BPRLoss(reg_lambda=1e-4)

epochs_lg = 200
refresh_every_lg = 5
n_samples_per_user_lg = 20
hard_ratio_lg = 0.3

loss_history_LightGCN = []
neighbors_idx_lg = None

print("Training LightGCN ...")
for epoch in range(epochs_lg):
    lightgcn.train()

    # cache embedding + neighbors cho hard negative
    with torch.no_grad():
        emb_cache = lightgcn(norm_adj_dev)
        item_emb_cache = emb_cache[num_users:]
        if (epoch == 0) or (epoch % refresh_every_lg == 0):
            neighbors_idx_lg = build_item_neighbors(item_emb_cache, topk=100, batch=1024)

    # negative sampling (mixed hard + random giống NGCF)
    users, pos_items, neg_items = mixed_negative_sampling(
        user_train_items,
        num_items,
        item_emb=item_emb_cache,
        neighbors_idx=neighbors_idx_lg,
        n_samples_per_user=n_samples_per_user_lg,
        mix=hard_ratio_lg,
    )
    users = users.to(device)
    pos_items = pos_items.to(device)
    neg_items = neg_items.to(device)

    emb = lightgcn(norm_adj_dev)
    u_emb = emb[:num_users]
    i_emb = emb[num_users:]

    user_batch_emb = u_emb[users]
    pos_emb = i_emb[pos_items]
    neg_emb = i_emb[neg_items]

    loss = bpr_lg(user_batch_emb, pos_emb, neg_emb)
    optimizer_lg.zero_grad()
    loss.backward()
    optimizer_lg.step()

    loss_history_LightGCN.append(loss.item())

    if (epoch + 1) % 5 == 0:
        lightgcn.eval()
        with torch.no_grad():
            emb_eval = lightgcn(norm_adj_dev)
            u_eval = emb_eval[:num_users]
            i_eval = emb_eval[num_users:]
            metrics_lightgcn = evaluate_embeddings(
                u_eval, i_eval, user_train_items, user_test_items, k=10
            )
        print(
            f"[LightGCN] Epoch {epoch+1:03d} | loss={loss.item():.4f} | "
            f"HR@10={metrics_lightgcn['HitRate@10']:.4f} | "
            f"Recall@10={metrics_lightgcn['Recall@10']:.4f} | "
            f"NDCG@10={metrics_lightgcn['NDCG@10']:.4f} | "
            f"Precision@10={metrics_lightgcn['Precision@10']:.4f} | "
            f"Users_eval={metrics_lightgcn['Users_eval']}"
        )

lightgcn.eval()
with torch.no_grad():
    emb_lg_final = lightgcn(norm_adj_dev)
    user_emb_lg = emb_lg_final[:num_users]
    item_emb_lg = emb_lg_final[num_users:]

print("Done LightGCN. user_emb_lg:", user_emb_lg.shape,
      "| item_emb_lg:", item_emb_lg.shape)

# Đánh giá LightGCN-only (lưu lại để tổng hợp báo cáo)
metrics_lightgcn = evaluate_embeddings(
    user_emb_lg, item_emb_lg,
    user_train_items, user_test_items,
    k=10
)
print("LightGCN only:", metrics_lightgcn)


In [ ]:
save_cf_model(
    "lightgcn",
    model=lightgcn,
    user_emb=user_emb_lg,
    item_emb=item_emb_lg,
    extra_info={
        "num_users": num_users,
        "num_items": num_items,
        "dim": lightgcn.dim,
        "n_layers": lightgcn.n_layers,
    }
)

# ALS

In [ ]:
# ================== ALS (Implicit 0/1) ==================
def train_als_implicit(A_csr, n_factors=64, n_iters=15, reg=0.1, seed=42):
    """
    A_csr: scipy.sparse.csr_matrix [num_users, num_items] với giá trị 0/1
    n_factors: số chiều latent
    n_iters: số vòng lặp ALS
    reg: hệ số regularization
    Trả về:
        P: user_factors [num_users, n_factors]
        Q: item_factors [num_items, n_factors]
    """
    rng = np.random.RandomState(seed)
    n_users, n_items = A_csr.shape

    P = 0.01 * rng.randn(n_users, n_factors).astype(np.float32)
    Q = 0.01 * rng.randn(n_items, n_factors).astype(np.float32)

    # dùng transpose để update item
    A_csc = A_csr.T.tocsr()

    I_f = np.eye(n_factors, dtype=np.float32)

    for it in range(n_iters):
        t0 = time.time()

        # ---- update user factors ----
        QTQ = Q.T @ Q      # [f, f]
        for u in range(n_users):
            start, end = A_csr.indptr[u], A_csr.indptr[u + 1]
            idx_items = A_csr.indices[start:end]
            if len(idx_items) == 0:
                continue
            Y = Q[idx_items]                 # [n_i, f]
            A = QTQ + reg * I_f + (Y.T @ Y - QTQ)   # ≈ Y.T @ Y + reg * I
            b = Y.T @ np.ones(len(idx_items), dtype=np.float32)
            P[u] = np.linalg.solve(A, b)

        # ---- update item factors ----
        PTP = P.T @ P
        for i in range(n_items):
            start, end = A_csc.indptr[i], A_csc.indptr[i + 1]
            idx_users = A_csc.indices[start:end]
            if len(idx_users) == 0:
                continue
            X = P[idx_users]                 # [n_u, f]
            A = PTP + reg * I_f + (X.T @ X - PTP)
            b = X.T @ np.ones(len(idx_users), dtype=np.float32)
            Q[i] = np.linalg.solve(A, b)

        print(f"[ALS] iter {it+1}/{n_iters} done in {time.time()-t0:.1f}s")

    return P, Q

In [ ]:
# Train ALS trên A_train
als_factors = 64
als_iters = 15
als_reg = 0.1

print("Training ALS (implicit 0/1) ...")
P_als, Q_als = train_als_implicit(
    A_train,
    n_factors=als_factors,
    n_iters=als_iters,
    reg=als_reg,
)

# Chuyển sang torch để dùng lại evaluate_embeddings
user_emb_als = torch.from_numpy(P_als).float().to(device)
item_emb_als = torch.from_numpy(Q_als).float().to(device)

print("ALS user_emb:", user_emb_als.shape,
      "| ALS item_emb:", item_emb_als.shape)

metrics_als = evaluate_embeddings(
    user_emb_als, item_emb_als,
    user_train_items, user_test_items,
    k=10
)
print("ALS only:", metrics_als)


In [ ]:
# user_emb_als, item_emb_als = torch.from_numpy(P), torch.from_numpy(Q)
save_cf_model(
    "als",
    model=None,                     # ALS không có state_dict
    user_emb=user_emb_als,
    item_emb=item_emb_als,
    extra_info={
        "num_users": user_emb_als.shape[0],
        "num_items": item_emb_als.shape[0],
        "dim": user_emb_als.shape[1],
    }
)

# Load model

In [41]:
lightgcn_loaded, user_emb_lg, item_emb_lg, info_lg = load_cf_model(
    "lightgcn",
    ModelClass=LightGCN,
    device=device
)
_, user_emb_als, item_emb_als, info_als = load_cf_model(
    "als",
    ModelClass=None,
    device=device
)


[OK] Loaded lightgcn from saved_models/lightgcn.pt
[OK] Loaded als from saved_models/als.pt


# CBFRS

In [47]:
# ==== CBF: Chuẩn bị content embedding cho item (từ CLIP) ====
import torch
import torch.nn.functional as F
import numpy as np

# item_feat: np.ndarray [num_items, feat_dim] đã build ở trên
# has_emb:   np.ndarray [num_items] bool

item_feat_t = torch.from_numpy(item_feat).float().to(device)   # [num_items, D]
item_feat_t = F.normalize(item_feat_t, p=2, dim=1)             # đảm bảo L2-norm

has_emb_t = torch.from_numpy(has_emb).to(device)               # [num_items]
print("CBF item_feat_t:", item_feat_t.shape, "| has_emb_t:", has_emb_t.shape)

CBF item_feat_t: torch.Size([10986, 512]) | has_emb_t: torch.Size([10986])


In [48]:
# ==== CBF: build user profile + recommend + evaluate (dùng chung train/test với CF) ====

def build_user_profile_cbf(u, user_train_items, item_feat_t, has_emb_t=None):
    """
    Xây profile nội dung cho user u bằng trung bình embedding CLIP
    của các item user đó đã tương tác (trong TRAIN).

    u: user_id đã encode (0..num_users-1)
    user_train_items: dict[u] -> list[item_id]
    item_feat_t: torch.Tensor [num_items, D] (đã normalize)
    has_emb_t: optional, torch.BoolTensor [num_items] (item nào có embedding)
    """
    items = user_train_items.get(u, [])
    if not items:
        return None

    items = torch.tensor(items, dtype=torch.long, device=item_feat_t.device)

    if has_emb_t is not None:
        mask = has_emb_t[items]
        items = items[mask]
        if items.numel() == 0:
            return None

    embs = item_feat_t[items]        # [n, D]
    prof = embs.mean(dim=0)          # [D]
    prof = F.normalize(prof, p=2, dim=0)
    return prof


def recommend_cbf_for_user_cf_idx(
    u,
    user_train_items,
    item_feat_t,
    has_emb_t=None,
    topk=10,
    exclude_seen=True,
):
    """
    Recommend THUẦN CBF cho user u (ID đã encode).
    Trả về list (item_id, score).
    """
    user_prof = build_user_profile_cbf(u, user_train_items, item_feat_t, has_emb_t)
    if user_prof is None:
        return []

    scores = item_feat_t @ user_prof   # [num_items], cosine vì đã normalize

    # loại các item đã thấy trong train nếu cần
    if exclude_seen:
        seen = set(user_train_items.get(u, []))
        if seen:
            seen_idx = torch.tensor(list(seen), dtype=torch.long, device=item_feat_t.device)
            scores[seen_idx] = -1e9

    # nếu có has_emb_t, loại item không có embedding
    if has_emb_t is not None:
        scores[~has_emb_t] = -1e9

    # top-k
    topk = min(topk, scores.shape[0])
    top_scores, top_idx = torch.topk(scores, k=topk)

    recs = [(int(top_idx[i].item()), float(top_scores[i].item())) for i in range(topk)]
    return recs


def evaluate_cbf_cf_idx(
    item_feat_t,
    user_train_items,
    user_test_items,
    has_emb_t=None,
    k=10,
):
    """
    Đánh giá CBF-only với cùng split train/test như CF.
    Trả về Precision@k, Recall@k, NDCG@k, HitRate@k.
    """
    item_feat_n = F.normalize(item_feat_t, p=2, dim=1)

    precisions, recalls, ndcgs, hit_rates = [], [], [], []

    for u in user_test_items.keys():
        test_items = set(user_test_items.get(u, []))
        if not test_items:
            continue

        user_prof = build_user_profile_cbf(u, user_train_items, item_feat_n, has_emb_t)
        if user_prof is None:
            continue

        scores = item_feat_n @ user_prof   # [num_items]

        train_items = set(user_train_items.get(u, []))
        if train_items:
            train_idx = torch.tensor(list(train_items), dtype=torch.long, device=item_feat_t.device)
            scores[train_idx] = -1e9

        if has_emb_t is not None:
            scores[~has_emb_t] = -1e9

        top_k = torch.topk(scores, k=min(k, scores.shape[0])).indices.cpu().numpy()
        hits = len(set(top_k) & test_items)

        precision = hits / k
        recall = hits / len(test_items)

        # NDCG
        dcg = 0.0
        for rank, iid in enumerate(top_k):
            if iid in test_items:
                dcg += 1.0 / np.log2(rank + 2)
        idcg = sum(1.0 / np.log2(i + 2) for i in range(min(len(test_items), k)))
        ndcg = dcg / idcg if idcg > 0 else 0.0

        hit = 1.0 if hits > 0 else 0.0

        precisions.append(precision)
        recalls.append(recall)
        ndcgs.append(ndcg)
        hit_rates.append(hit)

    return {
        f"Precision@{k}": float(np.mean(precisions)) if precisions else 0.0,
        f"Recall@{k}": float(np.mean(recalls)) if recalls else 0.0,
        f"NDCG@{k}": float(np.mean(ndcgs)) if ndcgs else 0.0,
        f"HitRate@{k}": float(np.mean(hit_rates)) if hit_rates else 0.0,
        "Users_eval": int(len(precisions)),
    }

In [49]:
metrics_cbf = evaluate_cbf_cf_idx(
    item_feat_t,
    user_train_items,
    user_test_items,
    has_emb_t=has_emb_t,
    k=10
)
print("CBF only:", metrics_cbf)

CBF only: {'Precision@10': 0.0019146608315098468, 'Recall@10': 0.004347886492306624, 'NDCG@10': 0.003707473113997233, 'HitRate@10': 0.019146608315098467, 'Users_eval': 1828}


In [50]:
# ==== Ensemble CF (LightGCN) + CBF trên cùng split train/test ====

def evaluate_cf_cbf_ensemble(
    user_emb_lg,          # user_emb_lg: [num_users, D_cf]
    item_emb_lg,          # item_emb_lg: [num_items, D_cf]
    item_feat_t,          # content embedding (text-only CBF): [num_items, D_cb]
    user_train_items,
    user_test_items,
    has_emb_t=None,
    alpha_cf=0.7,         # trọng số CF (LightGCN)
    alpha_cb=0.3,         # trọng số CBF
    k=10,
):
    # Chuẩn hóa embedding CF (LightGCN) và CBF
    U_cf = F.normalize(user_emb_lg, dim=1)        # [num_users, D_cf]
    I_cf = F.normalize(item_emb_lg, dim=1)        # [num_items, D_cf]
    I_cb = F.normalize(item_feat_t, dim=1)        # [num_items, D_cb]

    precisions, recalls, ndcgs, hit_rates = [], [], [], []

    for u in user_test_items.keys():
        test_items = set(user_test_items.get(u, []))
        if not test_items:
            continue

        # --- CF scores (LightGCN) ---
        scores_cf = (I_cf @ U_cf[u]).clone()      # [num_items]

        # --- CBF scores ---
        user_prof_cb = build_user_profile_cbf(u, user_train_items, I_cb, has_emb_t)
        if user_prof_cb is None:
            scores_cb = torch.zeros_like(scores_cf)
        else:
            scores_cb = I_cb @ user_prof_cb       # [num_items]

        # --- Ensemble ---
        scores = alpha_cf * scores_cf + alpha_cb * scores_cb

        # loại item đã thấy trong train
        train_items = set(user_train_items.get(u, []))
        if train_items:
            train_idx = torch.tensor(
                list(train_items), dtype=torch.long, device=item_feat_t.device
            )
            scores[train_idx] = -1e9

        # nếu chỉ một phần item có content embedding
        if has_emb_t is not None:
            scores[~has_emb_t] = -1e9

        # top-k
        top_k = torch.topk(scores, k=min(k, scores.shape[0])).indices.cpu().numpy()
        hits = len(set(top_k) & test_items)

        precision = hits / k
        recall = hits / len(test_items)

        # NDCG
        dcg = 0.0
        for rank, iid in enumerate(top_k):
            if iid in test_items:
                dcg += 1.0 / np.log2(rank + 2)
        idcg = sum(1.0 / np.log2(i + 2) for i in range(min(len(test_items), k)))
        ndcg = dcg / idcg if idcg > 0 else 0.0

        hit = 1.0 if hits > 0 else 0.0

        precisions.append(precision)
        recalls.append(recall)
        ndcgs.append(ndcg)
        hit_rates.append(hit)

    return {
        f"Precision@{k}": float(np.mean(precisions)) if precisions else 0.0,
        f"Recall@{k}": float(np.mean(recalls)) if recalls else 0.0,
        f"NDCG@{k}": float(np.mean(ndcgs)) if ndcgs else 0.0,
        f"HitRate@{k}": float(np.mean(hit_rates)) if hit_rates else 0.0,
        "Users_eval": int(len(precisions)),
    }


In [51]:
# ================== CF-only ==================
# NGCF
metrics_ngcf = evaluate_embeddings(
    user_emb_ngcf, item_emb_ngcf,
    user_train_items, user_test_items,
    k=10
)
print("NGCF only:", metrics_ngcf)

# LightGCN
metrics_lg = evaluate_embeddings(
    user_emb_lg, item_emb_lg,
    user_train_items, user_test_items,
    k=10
)
print("LightGCN only:", metrics_lg)

# ALS
metrics_als = evaluate_embeddings(
    user_emb_als, item_emb_als,
    user_train_items, user_test_items,
    k=10
)
print("ALS only:", metrics_als)


# ================== CBF-only ==================
metrics_cbf = evaluate_cbf_cf_idx(
    item_feat_t,
    user_train_items, user_test_items,
    has_emb_t=has_emb_t,
    k=10
)
print("CBF only:", metrics_cbf)


# ================== Ensemble CF + CBF ==================
# NGCF + CBF
metrics_ens_ngcf = evaluate_cf_cbf_ensemble(
    user_emb_ngcf, item_emb_ngcf,
    item_feat_t,
    user_train_items, user_test_items,
    has_emb_t=has_emb_t,
    alpha_cf=0.7,
    alpha_cb=0.3,
    k=10
)
print("Ensemble NGCF + CBF:", metrics_ens_ngcf)

# LightGCN + CBF
metrics_ens_lg = evaluate_cf_cbf_ensemble(
    user_emb_lg, item_emb_lg,
    item_feat_t,
    user_train_items, user_test_items,
    has_emb_t=has_emb_t,
    alpha_cf=0.7,
    alpha_cb=0.3,
    k=10
)
print("Ensemble LightGCN + CBF:", metrics_ens_lg)

# ALS + CBF
metrics_ens_als = evaluate_cf_cbf_ensemble(
    user_emb_als, item_emb_als,
    item_feat_t,
    user_train_items, user_test_items,
    has_emb_t=has_emb_t,
    alpha_cf=0.7,
    alpha_cb=0.3,
    k=10
)
print("Ensemble ALS + CBF:", metrics_ens_als)


NGCF only: {'Precision@10': 0.018161925601750548, 'Recall@10': 0.036449981619190996, 'NDCG@10': 0.028554664316951677, 'HitRate@10': 0.14387308533916848, 'Users_eval': 1828}
LightGCN only: {'Precision@10': 0.01849015317286652, 'Recall@10': 0.03866659575480487, 'NDCG@10': 0.03233667147485906, 'HitRate@10': 0.15590809628008753, 'Users_eval': 1828}
ALS only: {'Precision@10': 0.013785557986870896, 'Recall@10': 0.031093409672675053, 'NDCG@10': 0.024891640492193882, 'HitRate@10': 0.11214442013129103, 'Users_eval': 1828}
CBF only: {'Precision@10': 0.0019146608315098468, 'Recall@10': 0.004347886492306624, 'NDCG@10': 0.003707473113997233, 'HitRate@10': 0.019146608315098467, 'Users_eval': 1828}
Ensemble NGCF + CBF: {'Precision@10': 0.0028993435448577683, 'Recall@10': 0.005826424700727855, 'NDCG@10': 0.004444511168844332, 'HitRate@10': 0.027899343544857767, 'Users_eval': 1828}
Ensemble LightGCN + CBF: {'Precision@10': 0.005306345733041575, 'Recall@10': 0.012202881182853799, 'NDCG@10': 0.0086456464

In [ ]:
# Popularity: số lần xuất hiện trong A_train
item_pop = np.asarray(A_train.sum(axis=0)).flatten().astype(np.float32)
if item_pop.max() > 0:
    item_pop_norm = item_pop / (item_pop.max() + 1e-8)
else:
    item_pop_norm = np.zeros_like(item_pop, dtype=np.float32)

item_pop_t = torch.from_numpy(item_pop_norm).float().to(device)

In [ ]:
def recommend_popular(topk=10, exclude_items=None):
    scores = item_pop_t.clone()
    if exclude_items:
        idx = torch.tensor(list(exclude_items), dtype=torch.long, device=item_pop_t.device)
        scores[idx] = -1e9
    k = min(topk, scores.shape[0])
    top_scores, top_idx = torch.topk(scores, k=k)
    return [(int(top_idx[i].item()), float(top_scores[i].item())) for i in range(k)]

In [ ]:
def recommend_for_user(
    u,
    user_train_items,
    user_emb_cf,
    item_emb_cf,
    item_feat_t,
    has_emb_t=None,
    topk=10,
    min_interactions_cf=3,
):
    """
    u: user_id đã encode (0..num_users-1) và đã đi qua remap kept_users
    """

    # 1) Lấy history của user
    history = user_train_items.get(u, [])
    n_hist = len(history)

    # Nếu user không có trong train (trong thực tế online gặp, trong offline ít gặp)
    if n_hist == 0:
        # cold user hoàn toàn: recommend theo POP
        return recommend_popular(topk=topk, exclude_items=None)

    # 2) Xác định trọng số CF / CBF tùy theo độ dày history
    if n_hist < min_interactions_cf:
        # lịch sử mỏng: ưu tiên CBF
        alpha_cf = 0.3
        alpha_cb = 0.7
    else:
        # lịch sử đủ dày: CF mạnh hơn
        alpha_cf = 0.7
        alpha_cb = 0.3

    # 3) Chuẩn hóa embedding CF / CBF
    U_cf = F.normalize(user_emb_cf, dim=1)
    I_cf = F.normalize(item_emb_cf, dim=1)
    I_cb = F.normalize(item_feat_t, p=2, dim=1)

    # --- CF scores ---
    scores_cf = (I_cf @ U_cf[u]).clone()  # [num_items]

    # --- CBF scores ---
    user_prof_cb = build_user_profile_cbf(u, user_train_items, I_cb, has_emb_t)
    if user_prof_cb is None:
        scores_cb = torch.zeros_like(scores_cf)
    else:
        scores_cb = I_cb @ user_prof_cb

    # --- Ensemble CF + CBF ---
    scores = alpha_cf * scores_cf + alpha_cb * scores_cb

    # Mask các item đã xem
    seen = set(history)
    if seen:
        seen_idx = torch.tensor(list(seen), dtype=torch.long, device=item_feat_t.device)
        scores[seen_idx] = -1e9

    # Không dùng item không có content nếu muốn
    if has_emb_t is not None:
        scores[~has_emb_t] = -1e9

    # 4) Nếu user quá lạnh, có thể blend thêm POP
    # (option): scores += gamma * item_pop_t
    # Ví dụ cho user rất mới:
    if n_hist < 2:
        gamma = 0.2
        scores = scores + gamma * item_pop_t

    # top-k
    k = min(topk, scores.shape[0])
    top_scores, top_idx = torch.topk(scores, k=k)
    recs = [(int(top_idx[i].item()), float(top_scores[i].item())) for i in range(k)]
    return recs

In [ ]:
import random
i = random.randint(1, 1828)   
u_test = list(user_test_items.keys())[i]

recs = recommend_for_user(
    u_test,
    user_train_items,
    user_emb_ngcf,
    item_emb_ngcf,
    item_feat_t,
    has_emb_t=has_emb_t,
    topk=10,
    min_interactions_cf=3,
)
print("Hybrid recs for user", u_test, ":", recs[:5])

In [ ]:
results = {
    "NGCF": metrics_ngcf,
    "LightGCN": metrics_lg,
    "ALS": metrics_als,
    "CBF": metrics_cbf,
    "NGCF+CBF": metrics_ens_ngcf,
    "LightGCN+CBF": metrics_ens_lg,
    "ALS+CBF": metrics_ens_als,
}


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

metrics = ["Precision@10", "Recall@10", "NDCG@10", "HitRate@10"]
model_names = list(results.keys())

# tạo ma trận giá trị: rows = metrics, cols = models
values = np.array([[results[m][metric] for m in model_names] for metric in metrics])

plt.figure(figsize=(16, 8))

x = np.arange(len(model_names))
width = 0.18

for i, metric in enumerate(metrics):
    plt.bar(x + i*width, values[i], width, label=metric)

plt.xticks(x + width*1.5, model_names, rotation=45)
plt.ylabel("Score")
plt.title("So sánh các mô hình CF, CBF và Ensemble (k=10)")
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(16, 12))

for i, metric in enumerate(metrics):
    plt.subplot(2, 2, i+1)
    vals = [results[m][metric] for m in model_names]
    plt.bar(model_names, vals, color='skyblue')
    plt.title(metric)
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()
